In [ ]:
import pandas as pd

import pandas as pd
import alphalens as al

# 从CSV文件读取量价数据
file_path = "F:/quatntData/value_factor/price.csv"   # 文件的路径和文件名，注意：此处要换成你的文件地址
price_df = pd.read_csv(file_path, encoding='gbk', index_col=0) 
price_df['trade_date'] = pd.to_datetime(price_df['trade_date'])  # 将日期统一为datetime格式

# 计算波动率因子
price_df['STD_10'] = price_df.groupby('stock_code')['close'].rolling(10).std().reset_index(level=0, drop=True)
price_df['STD_100'] = price_df.groupby('stock_code')['close'].rolling(100).std().reset_index(level=0, drop=True)
price_df['STD_200'] = price_df.groupby('stock_code')['close'].rolling(200).std().reset_index(level=0, drop=True)

# 选取2010年之后的数据
df = price_df[price_df['trade_date']>='2010-01-01']  
df = df[['trade_date', 'stock_code', 'open', 'STD_10', 'STD_100', 'STD_200']].reset_index(drop=True)  

import pandas as pd
import alphalens as al
# 关闭警告信息
import warnings
warnings.filterwarnings('ignore')

# 生成符合Alphalens要求格式的因子值数据
factor_name ='STD_10'
factor = df.set_index(['trade_date','stock_code'])[factor_name]
factor

# 生成符合Alphalens要求格式的交易价格数据
prices = df.pivot(index='trade_date', columns='stock_code', values='open')
prices = prices.shift(-1)  # 将第二天的开盘价作为交易价格，避免用到“未来数据”

# 预处理因子数据，得到符合Alphalens需要的数据格式。
factor_data = al.utils.get_clean_factor_and_forward_returns(
    factor=factor,
    prices=prices,
    quantiles=10,
    periods=(1,))

# 计算因子分组收益
al.tears.create_returns_tear_sheet(factor_data)